In [1]:
%matplotlib inline
import utils_ted
from utils_ted import *

Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Setup

In [2]:
path = "../data/redux/"
model_path = path+"models/"
if not os.path.exists(model_path): os.makedirs(model_path)

In [3]:
batch_size=64

In [48]:
batches = get_batches(path+'train', batch_size=batch_size) 
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size) 

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [49]:
trn_classes, val_classes, trn_labels, val_labels, trn_filenames, val_filenames, test_filenames = get_classes(path) 

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In this notebook we're going to create an ensemble of models and use their average as our predictions. For each ensemble, we're going to follow our usual fine-tuning steps:

1) Create a model that retrains just the last layer
2) Add this to a model containing all VGG layers except the last layer
3) Fine-tune just the dense layers of this model (pre-computing the convolutional layers)
4) Add data augmentation, fine-tuning the dense layers without pre-computation.

So first, we need to create our VGG model and pre-compute the output of the conv layers:

In [ ]:
model = Vgg16().model
conv_layers, fc_layers = split_at(model, Conv2D)

In [ ]:
conv_model = Sequential(conv_layers)

In [51]:
conv_trn_features = conv_model.predict_generator()

Object `conv_model.predict_generator` not found.


In [ ]:
conv_trn_features = conv_model.predict_generator(batches, steps=ceil(batches.n/batches.batch_size), verbose=2)
conv_val_features = conv_model.predict_generator(val_batches, steps=ceil(valbatches.n/val_batches.batch_size), verbose=2)

In [ ]:
save_array(model_path + 'train_vgg16_convlayer_features.bc', conv_trn_features)
save_array(model_path + 'valid_vgg16_convlayer_features.bc', conv_val_features)

In the future we can just load these precomputed features:

In [ ]:
conv_trn_features = load_array(model_path + 'train_vgg16_convlayer_features.bc')
conv_val_features = load_array(model_path + 'valid_vgg16_convlayer_features.bc')

We can also save some time by pre-computing the training and validation arrays with the image decoding and resizing already done:

In [7]:
trn = get_data(path+'train')
val = get_data(path+'valid')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [8]:
save_array(model_path+'train_data.bc', trn)
save_array(model_path+'valid_data.bc', val)

In the future we can just load these resized images:

In [7]:
trn = load_array(model_path+'train_data.bc')
val = load_array(model_path+'valid_data.bc')

Finally, we can precompute the output of all but the last dropout and dense layers, for creating the first stage of the model:

In [ ]:
model.pop()
model.pop()

In [ ]:
ll_val_features = model.predict_generator()